### BOOK 만들기

#### (pycharm terminal)
```
python manage.py startapp book
```

#### config/settings.py
```config/settings.py
INSTALLED_APPS = [
    ...
    'book'
]
```

#### book/models.py
```book/models.py
from django.db import models

# Create your models here.
class Book(models.Model):
    idx = models.AutoField(primary_key = True)
    title = models.CharField(max_length=50, blank = True, null=True)
    author = models.CharField(max_length=20, blank=True, null=True)
    price = models.IntegerField(default = 0)
    amount = models.IntegerField(default=0)
```

#### 변경사항 반영 
```
python manage.py makemigrations   # DB에 반영할 사항 정리
python manage.py migrate          # DB에 반영
python manage.py runserver localhost:80
```

참고: book/migrations/0001_initail.py 에 기본 테이블이 있음

#### oracle sql developer
```
select * from tab;   # 테이블 목록
desc book_book;      # describe 테이블 구조 설명: 앱이름_클래스이름
```

#### book/urls.py
```book/urls.py
from django.urls import path
from book import views

urlpatterns = [
    path('', views.list),
    path('write', views.write),
    path('insert', views.insert),
    path('edit', views.edit),
    path('update', views.update),
    path('delete', views.delete),
]
```

#### config/urls.py
```config/urls.py
urlpatterns = [
    ...
    path('book/', include('book.urls')),
]
```

#### book/views.py
```book/views.py
from django.shortcuts import render, redirect
from book.models import Book
# Create your views here.
def list(request):
    bookList = Book.objects.order_by('-idx')
                # 전체레코드               -idx 내림차순 정렬
    return render(request, 'book/list.html', {'bookList': bookList, 'bookCount': len(bookList)})
                # book/list.html에 값을 채워서 출력해라

def write(request):
    return render(request, 'book/write.html')

def insert(request):
    book = Book(title = request.POST['title'],
                author = request.POST['author'],
                price = int(request.POST['price']),
                amount = int(request.POST['amount']))
            # get방식: 데이터요청   request.GET['변수명']
            # post방식: 데이터전달   request.POST['변수명']
    book.save()
    return redirect('/book')

def edit(request):
    row = Book.objects.get(idx = request.GET['idx'])
                # 글번호가 idx=5인 하나 선택
    return render(request, 'book/edit.html', {'row':row})

def update(request):
    book = Book(idx = request.POST['idx'],    # 글번호가 있으면 수정, 없으면 insert(글번호 자동)
                title = request.POST['title'],
                author = request.POST['author'],
                price = int(request.POST['price']),
                amount = int(request.POST['amount']))
    book.save()
    return redirect('/book')

def delete(request):
    Book.objects.get(idx = request.POST['idx']).delete()
    return redirect('/book')
```

#### address/templates/index.html
```address/templates/index.html
...
<a href="book/">도서관리</a><br>
...
```

#### book/templates/book/list.html
```book/templates/book/list.html
<!DOCTYPE html>
<!-- html 문서 버전, html 5-->
<html lang="en">
<head>  <!-- 문서 정보 -->
    <meta charset="UTF-8">
    <title>Title</title>
</head>
<body> <!-- 실제 내용 -->
    <h2>도서목록</h2>  <!-- h1~h6 -->
    등록된 도서: {{bookCount}}권  <!-- views의 변수명과 연결해서 봐야 함 -->
                <!-- {{변수}} : 변수에 저장된 값 출력 -->
    <input type="button" value="Home" onclick="location.href='/'">
                                    <!-- on+이벤트 -->
                                    <!-- onchange / onfocus / onblur ... -->
                                    <!-- location: 주소 객체,  href: 주소, 시작페이지 -->
    <input type="button" value="추가" onclick="location.href='/book/write'">

    <table border="1" width="600px">
        <tr>   <!-- table row 행 -->
            <td>제목</td> <!-- table division 열 -->
            <td>저자</td>
            <td>가격</td>
            <td>수량</td>
        </tr>

    {% for book in bookList %}
    <!-- for 개별 in 리스트(views의 값) -->
    <!-- 아래는 반복되는 템플릿 -->
        <tr>
            <td><a href="edit?idx={{book.idx}}">{{book.title}}</a></td>
            <td>{{book.author}}</td>
            <td>{{book.price}}</td>
            <td>{{book.amount}}</td>
        </tr>
    {% endfor %}
    </table>
</body>
</html>
```

#### book/templates/book/write.html
```book/templates/book/write.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
</head>
<body>
<h2>도서정보</h2>
<form method="post" action="insert">
  {% csrf_token %}
  <table border="1" width="500px">
    <tr>
      <td>제목</td>
      <td><input name="title" size="40"></td>
              <!-- title 변수명   받는 쪽 request.POST['title'] -->
    </tr>
    <tr>
      <td>저자</td>
      <td><input name="author"></td>
    </tr>
    <tr>
      <td>가격</td>
      <td><input type="number" name="price"></td>
    </tr>
    <tr>
      <td>수량</td>
      <td><input type="number" name="amount"></td>
    </tr>
    <tr align="center">
      <td colspan="2">
        <input type="submit" value="확인">
      </td>
    </tr>
  </table>
</form>

</body>
</html>
```

#### book/templates/book/edit.html
```book/templates/book/edit.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
  <script>
    function book_update(){
      document.form1.action = 'update';
      document.form1.submit();
    }
    function book_delete(){
      if(confirm("삭제하시겠습니까?")){
        document.form1.action = 'delete';
        document.form1.submit();
        }
      }
  </script>
</head>
<body>
<form name="form1" method="post">
  {% csrf_token %}
  <table border="1" width="500px">
    <tr>
      <td>제목</td>
      <td><input name="title" size="40" value="{{row.title}}"</td>
    </tr>
    <tr>
      <td>저자</td>
      <td><input name="author" value="{{row.author}}"></td>
    </tr>
    <tr>
      <td>가격</td>
      <td><input type="number" name="price" value="{{row.price}}"></td>
    </tr>
    <tr>
      <td>수량</td>
      <td><input type="number" name="amount" value="{{row.amount}}"></td>
    </tr>
    <tr align="center">
      <td colspan="2">
        <input type="hidden" name="idx" value="{{row.idx}}">
        <input type="button" value="수정" onclick="book_update()">
        <input type="button" value="삭제" onclick="book_delete()">
        <input type="button" value="목록" onclick="location.href='/book'">
      </td>
    </tr>
  </table>
</form>

</body>
</html>
```

#### 폴더 위치
- html파일을 각 앱의 templates 하위에 폴더 만든 후 그 하위에 생성해야함
- 장고는 위에서부터 읽어옴
- index.html은 address폴더가 아닌 다른 위치(config)의 templates폴더 하위에 있어도 무방함